# Exemplo: Modelo cinético de reação de isomerização

## Introdução


O problema abordado refere-se à estimação dos parâmetros do modelo cinético da reação de pirólise  do biciclohexano na fase de vapor. Esta reação tem como produto o 1,5-hexadieno [1] e é exemplificada pela Figura 1.

**Figura 1: Reação de pirólise do biciclohexano. Fonte: [1]**
![](./P.jpg)

Os dados experimentais para esta reação, numa faixa de temperatura de 327 à 366 ºC, são apresentados na Tabela 1.     

**Tabela 1: Dados experimentais. Fonte: [1]**     
                                               
  Temperatura / °C | Pressâo do reagente (mm) | Tempo (min) | Fração do reagente (%) |       $k$ x $10^{5}$
  :-------------: |  :----------------------: |:-----------:|:---------------------: |:------------------------:
         327.2    |         18.4                        |120              |90.0                  |1.46 
         327.2    |         18.3                        |60.0             |94.9                  |1.43
         338.9    |         18.3                        |60.0             |88.6                  |3.37
         338.9    |         18.1                        |120              |78.5                  |3.36
         338.9    |         18.9                        |120              |79.1                  |3.25
         338.9    |         18.6                        |60.0             |89.0                  |3.44
         338.9    |         22.5                        |60               |78.7                  |6.64  
         338.9    |         19.9                        |45.0             |69.5                  |13.5
         366.3    |         24.3                        |15.0             |80.8                  |23.6
           
           
O modelo cinético para esta reação é dado por [2]: 

$y = \exp\left[-K_0\cdot t \cdot \exp\left(-E(\frac{1}{T}\right))\right]$,                  (1)

em que $y$ é a fração do reagente, $t$ é o tempo, $T$ é a temperatura em K, por fim, $k_o$ e $E$ são os parâmetros a serem estimados.

Então, o problema de otimização a ser resolvido envolve o uso da função objetivo de mínimos quadrados ponderados pelo inverso da variância, conforme [2]:

$\min_{ko,E} \sum_{i=1}^{NE} \left(\frac{y^{exp}_i-y_i(k_o,E)}{u^2_{y_i}}\right)$ (2)

sujeito à (1).

Para resolução deste problema no MT_PEU serão usados os símbolos:

* Símbolos das grandezas independentes (tempo e temperatura): t, T
* Símbolos das grandezas dependentes (fração de reagente): y
* Símbolos dos parâmetros: ko, E

## Importação de pacotes

Importação de bibliotecas (pacotes) necessárias para execução do código.

* **MT_PEU**: biblioteca que contém as principais funcionalidades da ferramenta

    * Importar a classe **EstimacaoNaoLinear**, que será utilizada neste exemplo de estimação não linear

* **numpy**: biblioteca para computação científica

    * será necessária somente a função **exp** (exponencial) para construção do modelo

In [1]:
from IPython import display #Modificações futuras para a geração de gráficos
from ipywidgets.widgets.interaction import show_inline_matplotlib_plots#Modificações futuras para a geração de gráficos

from MT_PEU import EstimacaoNaoLinear
from numpy import exp

## Criação de modelo

O modelo (1) representa o comportamento da grandeza dependente, no qual os parâmetros $k_o$ e $E$ serão estimados.

Este modelo é definido, então, na forma de uma subrotina, **def**, do python e representado por:

In [2]:
def Modelo (param, x, args):

    ko, E = param[0], param[1]
    tempo, T = x[0], x[1] 
    
    return exp(-ko*tempo*exp(-E*(1/T-1./630.)))

## Inicialização de classe

Para executar a estimação, o primeiro passo é configurar a classe **EstimacaoNaoLinear**, inicialmente, incluindo informações básicas:

* O modelo,
* Os símbolos para as grandeza independentes *(x)*; 
* Os símbolos para as grandezas dependentes *(y)*; 
* Os símbolos para os parâmetros *(param)*;
* O nome do projeto, o nome da pasta onde serão gerados os resultados.


In [3]:
Estime = EstimacaoNaoLinear(Modelo, simbolos_x=[r't','T'], unidades_x=['s','K'], label_latex_x=[r'$t$','$T$'],
                            simbolos_y=[r'y'], unidades_y=['adm'],
                            simbolos_param=['ko','E'], unidades_param=['adm','K'],label_latex_param=[r'$k_o$',r'$E$'],
                            Folder='ModeloCineticoEX3')

## Inclusão de dados

Os dados experimentais disponibilizados em [2] da variável dependente (y) e das variáveis independentes (t e T), são apresentados abaixo na forma de listas:


In [4]:
#Tempo
tempo = [120.0,60.0,120.0,60.0,30.0,15.0,45.1,90.0,150.0,60.0,60.0,30.0,150.0,90.4,120.0,60.0,60.0,60.0,30.0,
         45.1,30.0,45.0,15.0,90.0,25.0,60.1,60.0,30.0,60.0]

#Temperatura
temperatura = [600.0,612.0,612.0,620.0,620.0,620.0,620.0,620.0,620.0,620.0,620.0,620.0,620.0,620.0,620.0,620.0,620.0,
               620.0,631.0,631.0,631.0,631.0,639.0,639.0,639.0,639.0,639.0,639.0,639.0]

y = [0.9,0.886,0.791,0.787,0.877,0.938,0.827,0.696,0.582,0.795,0.790,0.883,0.576,0.715,0.673,0.802,0.804,0.804,0.764,
     0.688,0.802,0.695,0.808,0.309,0.689,0.437,0.425,0.659,0.449]


Como entrada **obrigatória**, a plataforma MT_PEU necessita da **incerteza dos dados experimentais** (ux1, ux2, uy1).
Neste exemplo, foram adotados o valor 1 para as incertezas.

In [5]:
uxtempo = [0.001]*29
uxtemperatura = [0.05]*29
uy1 = [1]*29

**Inclusão de dados experimentais na estimação:**

O método *setDados* é utilizado para incluir os dados para as grandezas dependentes e independentes. Sintaxe:

* identificação da grandeza, se independente ou dependente: 0 ou 1 (respectivamente)
* os dados experimentais e suas incertezas inseridos em sequência na forma de tuplas.


In [6]:
Estime.setDados(0,(tempo,uxtempo),(temperatura,uxtemperatura))
Estime.setDados(1,(y,uy1))

Define que os dados experimentais previamente inseridos serão utilizados como um conjunto de dados para o qual os 
parâmetros serão estimados:

In [7]:
Estime.setConjunto()

## Otimização

O software permite utilizar diferentes algoritmos de otimização, a exemplo de: 'Nelder-Mead', 'Powell', 'BFGS', 'L-BFGS-B', 'CG'. 
Caso opte por não escolher, será utilizado o algoritmo default: Nelder-Mead.

Para realizar a otimização é obrigatório informar a estimativa inicial que, neste caso, será [0.005, 20000.000].


In [8]:
Estime.optimize(initial_estimative= [0.005, 20000.000],algoritmo='ipopt')



******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************



## Incerteza dos parâmetros

Como os dados experimentais possuem incerteza, estas devem ser propagadas para os parâmetros.

Métodos disponíveis: 2InvHessiana, Geral, SensibilidadeModelo. 

Para avaliação da região de abrangência dos parâmetros, pelo método da verossimilhança, a opção **preencherregiao**. 

Por definição o preenchimeto da região de verossimilhança é 'True', caso necessário esta opção pode ser alterada.

Caso o usuário **não** tenha interesse em **avaliar a incerteza dos parâmetros**, não execute o método abaixo.


In [10]:
Estime.incertezaParametros(metodoIncerteza='SensibilidadeModelo',preencherregiao=False)

Estime.graficos(["grandezas-entrada"]) #Mudanças futuras

<Figure size 432x288 with 0 Axes>

## Predição de análise de resíduos

 No método predição, é feita a avaliação da grandeza dependente com base nos parâmetros fornecidos. 
 A covariância é avaliada, e é possível inferir sobre a qualidade do modelo. 
 
 A análise de resíduos, tras mais informações referents à qualidade do modelo, a exemplo: (i) relação de tendência não inclusa no modelo, (ii) correlações indevidas, etc. Testes estatisticos são utilizados para estas avaliações como: (i) o de homocedasticidade, (ii) $\chi^2$.
 A predição e análise de residuos é feita com os dados de validação, quando disponíveis.

In [11]:
Estime.predicao()
Estime.analiseResiduos()

## Gráficos e relatório

Nesta etapa ocorre a geração dos dados de saída do programa : relátorios e gráficos. 
Os gráficos são gerados de acordo com as etapas que foram realizadas. No relátorio contém informações a respeito.
dos testes estatisticos, função objetivo, matriz de covariância, status da otimização, dentre outros.


In [12]:
etapas = ['otimizacao','grandezas-entrada', 'predicao','grandezas-calculadas','analiseResiduos', 'regiaoAbrangencia']
Estime.graficos(etapas)
Estime.relatorio()

/home/giufba-admin/Documentos/MT_PEU/MT_PEU.py:2018: UserWarning: Os gráficos de otimizacao não puderam ser criados, o algoritmo de otimização utilizado não possui gráficos de desempenho OU o método otimizacao não foi executado.
  warn('Os gráficos de otimizacao não puderam ser criados, o algoritmo de otimização utilizado não possui gráficos de desempenho OU o método otimizacao não foi executado.',UserWarning)


<Figure size 432x288 with 0 Axes>

<Figure size 360x240 with 0 Axes>

## Referências: 

[1] SRINIVASAN, R.; LEVI, A. A.. Kinetics of the Thermal Isomerization of Bicyclo [2.1.1 ]hexane. Journal Of The American Chemical Society, [s.l.], v. 85, n. 21, p.3363-3365, 5 nov. 1963. American Chemical Society (ACS)

[2] SCHWAAB, M.M.;PINTO, J.C. Análise de Dados Experimentais I: Fundamentos da Estátistica e Estimação de Parâmetros. 
Rio de Janeiro: e-papers, 2007.

INMETRO.: Avaliação de dados de medição — Guia para a expressão de incerteza de medição. Rio de Janeiro: Jcgm, 2008.